In [1]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [2]:
import msm_we
import numpy as np
import pickle
import ray

ray.init(num_cpus=2)  #Example if 2 cpus are used

2024-06-18 08:41:22,847	INFO worker.py:1749 -- Started a local Ray instance.


Python version:,3.10.14
Ray version:,2.23.0


In [3]:
import Coordinate_Processing
msm_we.modelWE.processCoordinates = Coordinate_Processing.processCoordinates

In [4]:
# Open the pickle file
with open('Outputs/SteadyState_Flux.obj', 'rb') as infile:
    msm_we_model = pickle.load(infile)

### One way to estimate the uncertainty of your model is to split your data into blocks, compute models (repeat the steps from get flux_matrix) over groups of the blocks, and assess consistency between the groups.

### The procedure here chops your data up into uniform blocks, by iteration. 
#### For example, with 100 iterations and 4 blocks, 
#### the blocks consist of iterations `[0-25), [25-50), [50-75), [75-100)`.
#### If the above blocks were assigned to 2 groups, the groups would consist of iterations `( [0-25), [50-75) )` and `(  [25-50), [75-100) )`

In [5]:
msm_we_model.do_block_validation(cross_validation_groups=2,cross_validation_blocks=4)

Block validation                    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:45
  Retrieving fluxmatrix tasks         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:16
  Retrieving discretized trajectories ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:06
  Retrieving fluxmatrix tasks         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
  Retrieving fluxmatrix tasks         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
  Retrieving discretized trajectories ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:07
  Retrieving fluxmatrix tasks         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00

#### Estimates from main model, all WE data are employed

In [6]:
rate_constant = msm_we_model.JtargetSS #Steady-State target flux/tau
print(f"The rate constant (using all WE data) for folding is {rate_constant}/s")

The rate constant (using all WE data) for folding is 611826.2247024674/s


### Estimates from block model: msm_we_model.validation_models[group index]
#### For eg: Steady state prob: cluster_pSS = msm_we_model.pSS (main model)
#### msm_we_model.validation_models[0].pSS

In [7]:
rate_constant = msm_we_model.validation_models[0].JtargetSS #Steady-State target flux/tau
print(f"The rate constant (estimate from first block) for folding is {rate_constant}/s")

rate_constant = msm_we_model.validation_models[1].JtargetSS #Steady-State target flux/tau
print(f"The rate constant (estimate from second block) for folding is {rate_constant}/s")

The rate constant (estimate from first block) for folding is 182993.96259687832/s
The rate constant (estimate from second block) for folding is 608838.4488528752/s


# Though this block validation provides an uncertainty estimate, the accurate way to estimate uncertainty in steady-state estimates is to perform replicate runs of WE simulations and compare the steady-state estimates between them.